In [1]:

from neo4j import GraphDatabase

class HelloWorldExample:

    def __init__(self, uri, user, password):
        self.driver = GraphDatabase.driver(uri, auth=(user, password))

    def close(self):
        self.driver.close()

    def print_greeting(self, message):
        with self.driver.session() as session:
            greeting = session.execute_write(self._create_and_return_greeting, message)
            print(greeting)

    @staticmethod
    def _create_and_return_greeting(tx, message):
        result = tx.run("CREATE (a:Greeting) "
                        "SET a.message = $message "
                        "RETURN a.message + ', from node ' + id(a)", message=message)
        return result.single()[0]


if __name__ == "__main__":
    greeter = HelloWorldExample("bolt://localhost:7687", "neo4j", "password")
    greeter.print_greeting("hello, world")
    greeter.close()

Transaction failed and will be retried in 1.188786698870791s (Couldn't connect to localhost:7687 (resolved to ()):
Failed to establish connection to ResolvedIPv6Address(('::1', 7687, 0, 0)) (reason [WinError 10061] No connection could be made because the target machine actively refused it)
Failed to establish connection to ResolvedIPv4Address(('127.0.0.1', 7687)) (reason [WinError 10061] No connection could be made because the target machine actively refused it))
Transaction failed and will be retried in 1.7092044937817767s (Couldn't connect to localhost:7687 (resolved to ()):
Failed to establish connection to ResolvedIPv6Address(('::1', 7687, 0, 0)) (reason [WinError 10061] No connection could be made because the target machine actively refused it)
Failed to establish connection to ResolvedIPv4Address(('127.0.0.1', 7687)) (reason [WinError 10061] No connection could be made because the target machine actively refused it))
Transaction failed and will be retried in 4.084588343352788s (C

ServiceUnavailable: Couldn't connect to localhost:7687 (resolved to ()):
Failed to establish connection to ResolvedIPv6Address(('::1', 7687, 0, 0)) (reason [WinError 10061] No connection could be made because the target machine actively refused it)
Failed to establish connection to ResolvedIPv4Address(('127.0.0.1', 7687)) (reason [WinError 10061] No connection could be made because the target machine actively refused it)

In [ ]:
!pip install neo4j

     ---------------------------------------- 0.0/192.8 kB ? eta -:--:--
     ------------------------------ ------- 153.6/192.8 kB 4.6 MB/s eta 0:00:01
     -------------------------------------- 192.8/192.8 kB 2.3 MB/s eta 0:00:00
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Installing backend dependencies: started
  Installing backend dependencies: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Created wheel for neo4j: filename=neo4j-5.14.1-py3-none-any.whl size=267587 sha256=67e636df23096e241a42d2f1960b4653aeb93bb2a94d2a8817acee2db5276cf2
  Stored in directory: c:\users\logan\appdata\local\pip\cache\wheels\da\1c\41\0330aeed780638bb8d232e94fffed164de968d778f8ebb3d23
Successfully built neo4j


In [ ]:
from neo4j import GraphDatabase

# create the uri for the connection
# uri = "neo4j://localhost:7687"
uri = "neo4j://localhost:7687/neo4j"
# create a driver object
driver = GraphDatabase.driver(uri, auth=("logan_vaz", "Bloom5791$5"))

# dir(driver)
print("connected: {}".format(driver.verify_connectivity()))

connected: None


In [ ]:
driver.execute_query("MATCH(n:Person) RETURN n").records
#great - we now have a database to store out information!

[<Record n=<Node element_id='4:014caa5e-007e-426d-8d25-2180fde12652:0' labels=frozenset({'Person', 'Actor'}) properties={'name': 'Charlie Sheen'}>>,
 <Record n=<Node element_id='4:014caa5e-007e-426d-8d25-2180fde12652:2' labels=frozenset({'Director', 'Person'}) properties={'name': 'Oliver Stone'}>>]

In [15]:
import praw 
reddit = praw.Reddit(
    client_id="54u8-WeUDusPlUoFQDZu4w",
    client_secret="lGhkiE6scCkSMYsj0P9AXzyYVSLBag",
    user_agent="jellyfish"
)

#reddit secret mD9JbzEVNFB8xopQKE_2WTcyIaz0hg, user_agen

In [18]:
ITERATION_NUM = 1

import time 
def get_redditor_data(redditor: praw.reddit):
    to_ret = dict()
    to_ret["username"] = redditor.name
    to_ret["comment_karma"] = redditor.comment_karma
    # to_ret["num_comments"] = len(redditor.comments.top())
    to_ret["time_creation"] = redditor.created_utc
    to_ret["verified_email"] = redditor.has_verified_email
    to_ret["is_employee"] = redditor.is_employee
    to_ret["is_mod"] = redditor.is_mod
    to_ret["is_gold"] = redditor.is_gold 
    to_ret["is_suspended"] = False
    to_ret["link_karma"] = redditor.link_karma
    to_ret["num_moderated"] = len(redditor.moderated())
    to_ret["num_multireddits"] = len(redditor.multireddits())
    to_ret["num_trophies"] = len(redditor.trophies())

    if (redditor.subreddit):
        to_ret["has_subreddit"] = True
        to_ret["over_18"] = redditor.subreddit.over_18#redditor.subreddit["over_18"]
        to_ret["num_subscribers"] = redditor.subreddit.subscribers#redditor.subreddit["subscribers"]

        #this is the only one added that rlly shouldn't be
        to_ret["public_description"] = redditor.subreddit.public_description
    else: to_ret["has_subreddit"] = False  
    
    to_ret = {"data": to_ret} 

    comment_data_user = list() 
    comment_data_dict = dict()
    for comment in redditor.comments.new():

        submission_id =comment.submission.id

        # print("Created date is " , comment.created_utc)
        comment_data_user.append([int(comment.created_utc), {"subreddit_id" : comment.subreddit_id, "submission_id": submission_id, "comment_id": comment.id}])

        comment_key = submission_id + "--------" + comment.id
        comment_data = {

            "data": {
                "is_edited": comment.edited,
                "link_title": comment.link_title, 
                "num_replies": len(comment.replies.list()),
                "score": comment.score, 
                "score_is_hidden": comment.score_hidden,
                "total_awards": comment.total_awards_received, 
                "num_ups": comment.ups, 
                "num_downs": comment.downs, 
                "body": comment.body,
                "date": comment.created_utc,
                "is_submitter": comment.is_submitter, 
                "stickied": comment.stickied
                
            },
            "meta": {
                 "ITERATION_NUM": ITERATION_NUM, 
                 "TIME": int(time.time()),
                 "submission_id": submission_id, 
                 "comment_id": comment.id,
                 "author": redditor.name
            }
            
        }
        comment_data_dict[comment_key] = comment_data
    
    to_ret["comments"] = comment_data_user 
    to_ret["meta"] = {
        "ITERATION_NUM": ITERATION_NUM, 
        "TIME": int(time.time())
    }

    return to_ret, comment_data_dict



# def get_submission_data():
    

In [12]:
redditor = reddit.redditor("SupremacyPlays")

In [14]:
redditor.id

AttributeError: 'Redditor' object has no attribute 'id'

In [27]:
!pip install --upgrade praw

In [32]:
import prawcore

In [36]:
#actually, I can usually run w/ csv bc not too many entries
import json 
import os

import traceback
import time    
epoch_time = int(time.time())

REDDITOR_FILE = "raw_data/redditor_dict.json"
SUBMISSION_FILE = "raw_data/submission_dict.json"
COMMENT_FILE = "raw_data/comment_dict.json"

def save_get_json(file_name):
    if (os.path.exists(file_name)):
        with open(file_name, 'r') as f:
            return json.load(f)
    return dict()

# with open("redditor_dict.json", 'r') as redditor_file:
#     redditor_dict = json.load(redditor_file)
# with open("submission_dict.json", 'r') as submission_file:
#     submission_dict = json.load(submission_file)
# with open("comment_dict.json", 'r') as comment_file:
#     comment_dict = json.load(comment_file)

redditor_dict = save_get_json(REDDITOR_FILE)
submission_dict = save_get_json(SUBMISSION_FILE)
comment_dict = save_get_json(COMMENT_FILE)

print("len of redditor_dict is ", len(redditor_dict))
print("len of submission_dict is ", len(submission_dict))
print("len of comment_dict is ", len(comment_dict))

time.sleep(1)

#first, let's just fill with the control group - by name
users_to_add = ['MoosieGoose', 'JollyK9', 'Southern_Ad3032', 'bduwowy272habbw', 'Late_Introduction203', 'kapster68', 'TheApertureMonkey', 'talemoon22', 'sebagolindenwald', 'spicyranchplzz', 'TheFloorMayBeLava_02', 'rxtten_flesh', 'greenblooded395', 'greenblooded395', 'DrakenJosh98', 'WhichUsernameIsBest', 'FStahp2', 'Pongpianskul', 'Kanashimi515', 'eviuwu', 'Kattheloner_22', 'Reeze2911', 'Sac20000', 'RanpoWasTaken', 'jlynny1811', 'Playful-Fail4778', 'GarageOk8109', 'katandcats', 'holyredemption', 'jifpeanutbutter420', 'Timely_Inflation1000', 'Erica_Peanut']
try:
    #get the basic user information 
    for user_name in users_to_add:
        print("USER IS ", user_name)
        if (user_name in redditor_dict.keys()): 
            print("skipping")
            time.sleep(1)
            continue
        # print("continuing")
        redditor =  reddit.redditor(user_name)
        user_data, comment_data = get_redditor_data(redditor) 

        #time to get all relevant submission data (TODO - this is how we'll add users - begging the question of the order to add them in but we'll deal w/ that later)
        this_submission_data = dict()
        cnt = 0 
        for comment in comment_data.values():

            cnt += 1
            print("comment cnt is ", cnt)
            submission_id = comment["meta"]["submission_id"] 
            if (submission_id in submission_dict or submission_id in this_submission_data): 
                print("matched id")
                continue 

            # print("submission id is " , submission_id)
            submission = reddit.submission(submission_id)

            #get comment data for this submission
            commentors = dict() 
            sub_comment_data = list()


            # print("len of comments is " , len(submission.comments.list()))

            def get_comments(listed):
                to_ret = list()
                for l in listed:
                    if (isinstance(l, praw.models.MoreComments)):
                        time.sleep(0.05)
                        try:
                            
                            base_comments = get_comments(l.comments())
                        except prawcore.exceptions.TooManyRequests as e:
                            print("backoff 1")
                            time.sleep(10)
                            try:
                                base_comments = get_comments(l.comments())
                            except prawcore.exceptions.TooManyRequests as e:
                                print("backoff 2")
                                time.sleep(30)
                                base_comments = get_comments(l.comments())
                        to_ret += base_comments 
                    else:
                        to_ret.append(l)
                return to_ret

            time.sleep(0.125)

            try:
                all_comments = submission.comments.list()
            except prawcore.exceptions.TooManyRequests as E:
                print("backoff 1")
                time.sleep(10)
                try:
                    all_comments = submission.comments.list()
                except prawcore.exceptions.TooManyRequests as E:
                    print("backoff 2")
                    time.sleep(30)
                    all_comments = submission.comments.list()
            

            original_comments = [c for c in all_comments if not isinstance(c, praw.models.MoreComments)]
            layered_comments = get_comments([c for c in all_comments if isinstance(c, praw.models.MoreComments)])

            for comment in original_comments:
                # submission.author != "None" or "[removed]" or "[deleted]"
                # print("comment author is ", comment.author)
                if (comment.author and comment.author not in ["None", "[removed]", "[deleted]"] and comment.author.name):
                    commentors[comment.author.name] = True
                    sub_comment_data.append( {
                        "user": comment.author.name,
                        "id": comment.id,
                        "original_reply": True
                    })

            # print("multicomment")
            for comment in layered_comments:
                # submission.author != "None" or "[removed]" or "[deleted]"
                print("comment author is ", comment.author)
                if (comment.author and comment.author not in ["None", "[removed]", "[deleted]"] and comment.author.name):
                    commentors[comment.author.name] = True
                    sub_comment_data.append( {
                        "user": comment.author.name,
                        "id": comment.id,
                        "original_reply": False
                    })
            # print("Dome w/ multi")
                
                

            #get submission data
            this_submission_data[submission_id] = {
                "created_utc": int(submission.created_utc),
                "distinguished": submission.distinguished,
                "is_original_content" : submission.is_original_content,
                "over_18": submission.over_18,
                "score": submission.score, 
                "title": submission.title,
                "edited": submission.edited, 
                "selftext": submission.selftext,
                "upvote_ratio": submission.upvote_ratio,
                "users_commented_on": list(commentors.keys()),
                "comment_data":sub_comment_data
            }

        print("updating dict")
        redditor_dict.update({user_name: user_data})
        submission_dict.update(this_submission_data)
        comment_dict.update(comment_data)

except Exception as E:
    traceback.print_exc()
    print("Exception is ", E)
    pass 
finally:
    with open(REDDITOR_FILE, 'w') as redditor_f:
        json.dump(redditor_dict,redditor_f )
    with open(SUBMISSION_FILE, 'w') as submission_f:
        json.dump(submission_dict, submission_f) 
    with open(COMMENT_FILE, 'w') as comment_f:
        json.dump(comment_dict,comment_f )
    print("Saved files")

"""
Redditor Instantiation
- load all data into memory first (static) 

get_base_data(): return redditor[data]
get_comment_data()
get_submission_data() 
//some kind of store value w/ state (seperate dict)
"""



"""
//all should contain stuff like a date scraped
Redditor_Dict:
Username ->  {
    "data": properties + labels in dict 
    "comments": [
        (comment date, {"subreddit_id" : subreddit_id, "submission_id": submission_id, "comment_id": comment_id})
        # (comment date, subreddit_id, .submission.id, {dictionary of all info})
        //dict of all info should be initialized seperately
    ]
}


[properties + labels below, comments: {relationship data + ["post_id" :id of post commented on [comment.subreddit_id]]}]
#what relevant edge data do we need?


SubmissionDict
{
    submission.id {
        "created_utc"
        "distinguished" 
        "is_original_content" 
        "over_18" 
        "score" 
        "title" 
        "upvote_ratio" 
        "users_commented_on": submission.comments.replace_more(None).list()
        "comment_data": {"user": comment.author.id, "id": comment.id}
    }
}

CommentDict {
    submission_id + "--------" + comment_id : { 
        all comment data
    }

}

"""


"""
end
"""

len of redditor_dict is  29
len of submission_dict is  1027
len of comment_dict is  1516
USER IS  MoosieGoose
skipping
USER IS  JollyK9
skipping
USER IS  Southern_Ad3032
skipping
USER IS  bduwowy272habbw
skipping
USER IS  Late_Introduction203
skipping
USER IS  kapster68
skipping
USER IS  TheApertureMonkey
skipping
USER IS  talemoon22
skipping
USER IS  sebagolindenwald
skipping
USER IS  spicyranchplzz
skipping
USER IS  TheFloorMayBeLava_02
skipping
USER IS  rxtten_flesh
skipping
USER IS  greenblooded395
skipping
USER IS  greenblooded395
skipping
USER IS  DrakenJosh98
skipping
USER IS  WhichUsernameIsBest
skipping
USER IS  FStahp2
skipping
USER IS  Pongpianskul
skipping
USER IS  Kanashimi515
skipping
USER IS  eviuwu
skipping
USER IS  Kattheloner_22
skipping
USER IS  Reeze2911
skipping
USER IS  Sac20000
skipping
USER IS  RanpoWasTaken
skipping
USER IS  jlynny1811
skipping
USER IS  Playful-Fail4778
skipping
USER IS  GarageOk8109
skipping
USER IS  katandcats
skipping
USER IS  holyredemption

In [ ]:
"""
options
- sleep
- batching 
- multiple user agents 
- scraping subseciton of comments
"""

In [ ]:
"""
Redditor:
{iteration, date of update, all labels below}

In [ ]:
#lets define what we want a redditor node to look like



#property - just need to return it
#label - if need to return it
class Redditor:
    """
    class to instantiate a Redditor node in our db

    properties:
        ITERATION_ITERATION_NUM
        
        iteration=ITERATION
        dateUpdated=TODAY()
        group=UNKNOWN (can update later to control vs treat)
        #do you return kind of id? I'd say no and we can do group search

    label:
        (copied over from user base data)
            
            to_ret["username"] = redditor.name
            to_ret["comment_karma"] = redditor.comment_karma
            to_ret["num_comments"] = len(redditor.comments)
            to_ret["time_creation"] = redditor.created_utc
            to_ret["verified_email"] = redditor.has_verified_email
            to_ret["is_employee"] = redditor.is_employee
            to_ret["is_mod"] = redditor.is_mod
            to_ret["is_gold"] = redditor.is_gold 
            to_ret["is_suspended"] = False
            to_ret["link_karma"] = redditor.link_karma
            to_ret["num_moderated"] = len(redditor.moderated())
            to_ret["num_multireddits"] = len(redditor.multireddits())
            to_ret["num_trophies"] = len(redditor.trophies())

            if (redditor.subreddit):
                to_ret["has_subreddit"] = True
                to_ret["over_18"] = redditor.subreddit.over_18#redditor.subreddit["over_18"]
                to_ret["num_subscribers"] = redditor.subreddit.subscribers#redditor.subreddit["subscribers"]

                #this is the only one added that rlly shouldn't be
                to_ret["public_description"] = redditor.subreddit["public_description"]
            else: to_ret["has_subreddit"] = False        

    relationships: type: COMMENTED_ON (connects to a POST class we still have to create)

            "is_edited": comment.edited,
            "link_title": comment.link_title, 
            "num_replies": len(comment.replies.list()),
            "score": comment.score, 
            "score_is_hidden": comment.score_hidden,
            "total_awards": comment.total_awards_received, 
            "num_ups": comment.ups, 
            "num_downs": comment.downs, 
            "body": comment.body,
            "date": comment.created_utc,
            "is_submitter": comment.is_submitter, 
            "stickied": comment.stickied

    """
    def __init__(self,):
        pass